<a href="https://colab.research.google.com/github/tylerfeldman321/GANs-for-Medical-Data/blob/main/notebooks/ECG_Synthesis_and_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline

In [ ]:
import os
%cd '/content/drive/MyDrive/MUSER'
if not os.path.exists('ECG-Synthesis-and-Classification'):
    !git clone https://github.com/mandrakedrink/ECG-Synthesis-and-Classification.git

In [ ]:
%cd '/content/drive/MyDrive/MUSER/ECG-Synthesis-and-Classification/ecg_gan'
import pandas as pd

dataset = 'wesad'

if dataset == 'mitbih':
    train_path = 'mitbih_train.csv'
    test_path = 'mitbih_test.csv'
elif dataset == 'wesad':
    train_path = 'wesad_train.csv'
    test_path = 'wesad_test.csv'

# df_ptbdb = pd.read_csv('ptbdb_abnormal.csv')
df_dataset = pd.read_csv(train_path)

In [ ]:
df_train = pd.read_csv(train_path, header=None)
df_test = pd.read_csv(test_path, header=None)
df_dataset = pd.concat([df_train, df_test], axis=0)
df_dataset.rename(columns={len(df_dataset.columns)-1: 'class'}, inplace=True)

id_to_label = {
    0: "Normal",
    1: "Artial Premature",
    2: "Premature ventricular contraction",
    3: "Fusion of ventricular and normal",
    4: "Fusion of paced and normal"
}
df_dataset['label'] = df_dataset.iloc[:, -1].map(id_to_label)
print(df_dataset.info())

In [ ]:
df_dataset.head()

In [ ]:
df_dataset.to_csv('data.csv', index=False)

In [ ]:
df_dataset['label'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
indexes = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
for index in indexes:
    print(df_dataset.iloc[index, -2:])
    row = df_dataset.iloc[index, :-2]
    row.plot(label=f'ECG Sample {index}')
    plt.xlabel('Time Index')
    plt.ylabel('Magnitude')
    plt.title('ECG Samples')

In [ ]:
!python train.py